## Importando as bibliotecas

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from pandas_profiling import ProfileReport
import sweetviz as sv

from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer

#### Configurando setup de visualização

In [2]:
%matplotlib inline
from IPython.core.pylabtools import figsize
figsize(16, 10)
sns.set()

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

## Leitura dos arquivos que compõe o dataset

### Base com informações sobre as empresas do Mercado a ser considerado.

In [3]:
market = pd.read_csv('estaticos_market.csv', sep=',', index_col=0)

### Portfólio empresa 1

In [4]:
port_1 = pd.read_csv('estaticos_portfolio1.csv', usecols=['id']).merge(market, on='id', how='inner')

### Portfólio empresa 2

In [5]:
port_2 = pd.read_csv('estaticos_portfolio2.csv', index_col=0).merge(market, on='id', how='inner')

### Ids dos clientes da empresa 3

In [6]:
port_3 = pd.read_csv('estaticos_portfolio3.csv', index_col=0).merge(market, on='id', how='inner')

### Funções criadas para auxiliar a resolução do problema

In [7]:
def exploracao(df):
    """
    Gera uma dataframe com informações relevantes sobre o mesmo.
    
    :param df: pd.DataFrame para ser realizada a exploração
    """
    exploracao = pd.DataFrame({
        'coluna' : df.columns,
        'tipos' : df.dtypes,
        'qtd_null' : df.isnull().sum()
    })
    
    exploracao['percent_null'] = np.round((exploracao.qtd_null / df.shape[0]) * 100, 4)
    
    return exploracao.sort_values(by='qtd_null', ascending=False)

In [8]:
def remover_colunas():
    
    # Razões para excluir
    
    # Alta cardinalidade
    # 'de_natureza_juridica', 'dt_situacao', 'nm_divisao', 'nm_micro_regiao'
    
    # Como aparenta ser uma coluna atualizada mês a mês, existe uma probabilidade de estar desatualizada
    # 'nu_meses_rescencia', 'de_saude_rescencia'
    
    # Existe coluna com esses dados de forma numerica, por esse motivo também vamos excluir estas
    # 'de_faixa_faturamento_estimado_grupo', 'de_faixa_faturamento_estimado', 'idade_emp_cat', 'fl_veiculo'
    
    # Colunas sem relevancia
    # 'fl_email', 'fl_telefone'
    
    # A matriz da empresa acredito que não vá ter relevância, então mantendo apenas a filial
    # sg_uf_matriz
    
    # Coluna com apenas valores false
    # fl_epp
    
    
    colunas = ['de_natureza_juridica', 'dt_situacao', 'nm_divisao', 'nm_micro_regiao', 'nu_meses_rescencia',
               'de_saude_rescencia', 'de_faixa_faturamento_estimado_grupo', 'de_faixa_faturamento_estimado',
               'idade_emp_cat', 'fl_veiculo', 'fl_email', 'fl_telefone', 'sg_uf_matriz', 'fl_epp' ]
    
    return colunas

In [9]:
def filter_columns(df):
    """
    Retorna um pd.DataFrame com as features que tem 80% de valores não nulos.
    
    :param df: pd.DataFrame para ser realizada a filtragem das features
    """
    df_exploracao = exploracao(df)
    features = df_exploracao[(df_exploracao.percent_null < 20.0)]['coluna']
    df = df[features]
    
    
    df = df[df.columns.difference(remover_colunas())]

    return df

In [10]:
# def encoding_obj(df, le):
#     """
#     Retorna pd.DataFrame com as colunas 'object' e 'booleanas' realizado o encoding
#     """
#     colunas_obj = df.columns[df.dtypes == object]
#     df[colunas_obj] = df[colunas_obj].apply(lambda col: le.fit_transform(col))
    
#     colunas_bool = df.columns[df.dtypes == bool]
#     df[colunas_bool] = df[colunas_bool].apply(lambda col: le.fit_transform(col))

#     return df

In [11]:
def treino_teste_port(df):
    X_train, X_test, y_train, y_test = train_test_split(
    df, list(range(len(df))), test_size=0.4, random_state=1)
    
    return X_train, X_test

In [12]:
def lista_simples(lista):
    if isinstance(lista, list):
        return [sub_elem for elem in lista for sub_elem in lista_simples(elem)]
    else:
        return [lista]

In [13]:
def probabilidade(x, divisor):
    resultado = np.round((x * 100) / divisor, 2)
    
    print(f'A probabilidade de acerto foi de: {resultado}%')

In [14]:
def fillna_columns(df):
    for col in df.columns:
        
        if 'fl_' in col:
            if {'SIM', 'NAO'} <= set(df[col]):
                df[col] = (df[col] == 'SIM')
            df[col] = df[col].fillna(False)  
            
        elif df[col].dtype == 'object':
            df[col] = df[col].fillna('SEM INFORMACAO' if 'de_' in col else 'OUTRA')
            
        elif df[col].dtype == 'float':
            mean = df[col].mean()
            df[col] = df[col].fillna(mean)

        else:
             pass
            
    return df

## Conhecendo mais sobre os dados

In [15]:
exploracao(market)

,coluna,tipos,qtd_null,percent_null
qt_alteracao_socio_total,qt_alteracao_socio_total,float64,462298,100.0000
qt_alteracao_socio_90d,qt_alteracao_socio_90d,float64,462298,100.0000
grau_instrucao_macro_desconhecido,grau_instrucao_macro_desconhecido,float64,462298,100.0000
qt_alteracao_socio_180d,qt_alteracao_socio_180d,float64,462298,100.0000
qt_alteracao_socio_365d,qt_alteracao_socio_365d,float64,462298,100.0000
coligada_mais_antiga_baixada,coligada_mais_antiga_baixada,float64,462162,99.9706
coligada_mais_nova_baixada,coligada_mais_nova_baixada,float64,462162,99.9706
idade_media_coligadas_baixadas,idade_media_coligadas_baixadas,float64,462162,99.9706
vl_total_veiculos_antt,vl_total_veiculos_antt,float64,462122,99.9619
vl_total_tancagem,vl_total_tancagem,float64,462018,99.9394


In [16]:
exploracao(port_1)

,coluna,tipos,qtd_null,percent_null
vl_total_veiculos_antt_grupo,vl_total_veiculos_antt_grupo,float64,555,100.0000
idade_media_coligadas_baixadas,idade_media_coligadas_baixadas,float64,555,100.0000
vl_total_tancagem_grupo,vl_total_tancagem_grupo,float64,555,100.0000
coligada_mais_antiga_baixada,coligada_mais_antiga_baixada,float64,555,100.0000
coligada_mais_nova_baixada,coligada_mais_nova_baixada,float64,555,100.0000
qt_alteracao_socio_total,qt_alteracao_socio_total,float64,555,100.0000
qt_alteracao_socio_90d,qt_alteracao_socio_90d,float64,555,100.0000
vl_total_veiculos_antt,vl_total_veiculos_antt,float64,555,100.0000
vl_total_tancagem,vl_total_tancagem,float64,555,100.0000
qt_alteracao_socio_180d,qt_alteracao_socio_180d,float64,555,100.0000


In [17]:
exploracao(port_2)

,coluna,tipos,qtd_null,percent_null
qt_alteracao_socio_180d,qt_alteracao_socio_180d,float64,566,100.0000
qt_alteracao_socio_total,qt_alteracao_socio_total,float64,566,100.0000
qt_alteracao_socio_365d,qt_alteracao_socio_365d,float64,566,100.0000
grau_instrucao_macro_desconhecido,grau_instrucao_macro_desconhecido,float64,566,100.0000
qt_alteracao_socio_90d,qt_alteracao_socio_90d,float64,566,100.0000
coligada_mais_nova_baixada,coligada_mais_nova_baixada,float64,565,99.8233
coligada_mais_antiga_baixada,coligada_mais_antiga_baixada,float64,565,99.8233
idade_media_coligadas_baixadas,idade_media_coligadas_baixadas,float64,565,99.8233
qt_socios_st_suspensa,qt_socios_st_suspensa,float64,565,99.8233
vl_total_veiculos_antt_grupo,vl_total_veiculos_antt_grupo,float64,561,99.1166


In [18]:
exploracao(port_3)

,coluna,tipos,qtd_null,percent_null
vl_total_veiculos_antt_grupo,vl_total_veiculos_antt_grupo,float64,265,100.0000
qt_alteracao_socio_90d,qt_alteracao_socio_90d,float64,265,100.0000
qt_alteracao_socio_365d,qt_alteracao_socio_365d,float64,265,100.0000
vl_total_veiculos_antt,vl_total_veiculos_antt,float64,265,100.0000
qt_alteracao_socio_180d,qt_alteracao_socio_180d,float64,265,100.0000
qt_socios_st_suspensa,qt_socios_st_suspensa,float64,265,100.0000
qt_alteracao_socio_total,qt_alteracao_socio_total,float64,265,100.0000
grau_instrucao_macro_desconhecido,grau_instrucao_macro_desconhecido,float64,265,100.0000
coligada_mais_nova_baixada,coligada_mais_nova_baixada,float64,264,99.6226
coligada_mais_antiga_baixada,coligada_mais_antiga_baixada,float64,264,99.6226


### Dicionário dos dados

<ol>
    <li><b>fl_matriz:</b> valor bool, verdadeiro se o CNPJ for "matriz" </li>
    <li><b>de_natureza_juridica:</b> caráter, natureza jurídica da empresa</li>
    <li><b>sg_uf:</b> caractere, iniciais do nome do estado da empresa</li>
    <li><b>natureza_juridica_macro:</b> uma categoria macro legal para "natureza jurídica"</li>
    <li><b style="color:blue;">de_ramo:</b> descrição de uma categoria / ramificação de atividade de macro com base no código CNAE.</li>
    <li><b style="color:blue;">setor:</b> caráter, setor econômico da empresa (Indústria, Serviços, Agronegócios, Etc.)</li>
    <li><b>idade_empresa_anos:</b> valor numérico, idade da empresa.</li>
    <li><b>idade_emp_cat:</b> caractere, idade da empresa por faixas</li>
    <li><b>fl_me:</b> valor booleano, verdadeiro se a empresa tiver o termo 'ME' no final de seu nome legal.</li>
    <li><b>fl_sa:</b> valor booleano, verdadeiro se a empresa tiver o termo 'SA' no final de seu nome legal</li>
    <li><b>fl_epp:</b> valor booleano, verdadeiro se a empresa tiver o termo 'EPP' no final de seu nome legal.</li>
    <li><b>fl_mei:</b> valor booleano, verdadeiro se a empresa tiver o termo 'MEI' no final de seu nome legal.</li>
    <li><b>fl_ltda:</b> valor booleano, verdadeiro se a empresa tiver o termo 'LTDA' no final de seu nome legal.</li>
    <li><b>dt_situacao:</b> data em que a "de_situacao" foi registrada pelo IRS</li>
    <li><b>fl_st_especial:</b>  valor booleano, verdadeiro se dt_situacao_especial não for nulo. Se não for nulo, significa que alguns situação extraordinária é identificada pelo IRS</li>
    <li><b>fl_email:</b> valor booleano, true se o cnpj tiver um email registrado no banco de dados do IRF</li>
    <li><b>fl_telefone:</b> valor booleano, verdadeiro se a empresa tiver um número de telefone registrado no banco de dados do IRS</li>
    <li><b>fl_rm:</b> caractere, verdadeiro se o endereço da empresa estiver localizado em uma área metropolitana.</li>
    <li><b>nm_divisao:</b> caractere, descrição do nome da atividade econômica primária da empresa</li>
    <li><b style="color:blue;">nm_segmento:</b> caractere, descrição do nome do cluster de atividades econômicas principais da empresa</li>
    <li><b>fl_spa:</b> booleano, verdadeiro se a empresa tiver seu próprio posto de combustível</li>
    <li><b>fl_antt:</b> boolean, true se a empresa é certificada pela ANTT para fins de transporte</li>
    <li><b>fl_veiculo:</b> booleano, verdadeiro se a empresa possuir pelo menos um veículo</li>
    <li><b style="color:#EB8958">vl_total_tancagem:</b> capacidade numérica e total de armazenamento de combustível da empresa</li>
    <li><b style="color:#EB8958">vl_total_veiculos_antt:</b> número inteiro, número total de veículos da empresa</li>
    <li><b>vl_total_veiculos_leves:</b> número inteiro, número total de veículos leves da empresa</li>
    <li><b>vl_total_veiculos_pesados:</b> número inteiro, número total de veículos pesados ​​da empresa</li>
    <li><b>fl_optante_simples:</b> booleano, verdadeiro se a empresa for tributada de acordo com o regime SIMPLES</li>
    <li><b>qt_art:</b> número inteiro, número de construções (edifícios) autorizadas por um arquiteto</li>
    <li><b>vl_total_veiculos_pesados_grupo:</b> valor econômico numérico dos veículos pesados ​​do grupo econômico da empresa</li>
    <li><b>vl_total_veiculos_leves_grupo:</b> valor econômico numérico dos veículos leves do grupo econômico da empresa</li>
    <li><b style="color:#EB8958">vl_total_tancagem_grupo:</b> capacidade numérica total de armazenamento de combustível do grupo econômico da empresa</li>
    <li><b style="color:#EB8958">vl_total_veiculos_antt_grupo:</b> numérico, número total de veículos do grupo econômico da empresa que está registrado com certificação ANTT</li>
    <li><b>vl_potenc_cons_oleo_gas:</b> consumo numérico e potencial de petróleo e gás</li>
    <li><b>fl_optante_simei:</b> booleano, verdadeiro se a empresa for tributada como MEI</li>
    <li><b>sg_uf_matriz:</b> caractere, estado do local do escritório principal</li>
    <li><b>de_saude_tributaria:</b> caractere, indicador do status do imposto sobre saúde, Verde se todos os impostos estiverem OK, Vermelho se nenhum estiver OK</li>
    <li><b>de_saude_rescencia:</b> caractere, indica o tempo de atualização da entrada mais atrasada do indicador.</li>
    <li><b>nu_meses_rescencia:</b> caractere, número de meses desde a última atualização da entrada mais atrasada de saude_tributaria</li>
    <li><b>de_nivel_atividade:</b> caractere, probabilidade de estar em operação, ALTA com alta probabilidade, BAIXA, baixa probabilidade.</li>
    <li><b>de_indicador_telefone:</b> caractere, propabilidade para chegar à empresa usando os números de telefone fornecidos por
Neoway</li>
    <li><b>fl_simples_irregular:</b> boolean, true se a empresa é tributada pelo regime SIMPLES e possui uma CNAE impeditiva (atividades econômicas que não podem ser tributadas de acordo com o regime simples)
receita acima do limite do regime.</li>
    <li><b>vl_frota:</b> valor numérico agregado de todos os veículos da empresa</li>
    <li><b>empsetorcensitariofaixarendapopulacao:</b> renda média numérica de uma amostra de residentes em uma unidade censitária (menor área territorial de acordo com o IBGE)</li>
    <li><b>nm_meso_regiao:</b> caractere, nome da região meso em que a empresa está localizada.</li>
    <li><b>nm_micro_regiao:</b> caractere, nome da microrregião em que a empresa está localizada.</li>
    <li><b>fl_passivel_iss:</b> booleano, true se a empresa realizar alguma atividade tributável no ISS - Imposto sobre serviços.</li>
    <li><b>qt_socios:</b> valor inteiro, quantidade de parceiros / acionistas do cnpj</li>
    <li><b>qt_socios_pf:</b> valor inteiro, quantidade de parceiros / acionistas da cnpj que são pessoas
q</li>
    <li><b>qt_socios_pj:</b> valor inteiro, quantidade de parceiros / acionistas da cnpj que são empresas</li>
    <li><b>idade_media_socios:</b> valor numérico, idade média dos parceiros do cnpj (considera apenas os parceiros do cpf)</li>
    <li><b>idade_maxima_socios:</b> igual ao anterior, mas com métrica máxima</li>
    <li><b>idade_minima_socios:</b> igual ao anterior, mas com métrica mínima</li>
    <li><b>qt_socios_st_regular:</b> valor inteiro, quantidade de parceiros com situação regular no IRS - Receita Federal</li>
    <li><b>qt_socios_st_suspensa:</b> valor inteiro, quantidade de parceiros com situação suspensa no IRS - Receit Federal</li>
    <li><b>qt_socios_masculino:</b> valor inteiro, quantidade de parceiros masculinos</li>
    <li><b>qt_socios_feminino:</b> valor inteiro, quantidade de parceiras (mulheres)</li>
    <li><b>qt_socios_pep:</b> valor inteiro, quantidade de parceiros do cnpj expostos politicamente</li>
    <li><b style="color:red;">qt_alteracao_socio_total:</b> valor inteiro, quantidade de alterações no QSA</li>
    <li><b style="color:red;">qt_alteracao_socio_90d:</b>  valor inteiro, quantidade de alterações no QSA nos últimos 90 dias</li>
    <li><b style="color:red;">qt_alteracao_socio_180d:</b>  valor inteiro, quantidade de alterações no QSA nos últimos 180 dias</li>
    <li><b style="color:red;">qt_alteracao_socio_365d:</b>  valor inteiro, quantidade de alterações no QSA nos últimos 365 dias</li>
    <li><b>qt_socios_pj_ativos:</b> valor inteiro, quantidade de parceiros PJ que são ATIVA no RF.</li>
    <li><b>qt_socios_pj_nulos:</b> valor inteiro, quantidade de parceiros PJ que são NULA no RF</li>
    <li><b>qt_socios_pj_baixados:</b> valor inteiro, quantidade de parceiros PJ que são BAIXADA no RF.</li>
    <li><b>qt_socios_pj_suspensos:</b> valor inteiro, quantidade de parceiros PJ que são SUSPENSA no RF.</li>
    <li><b>qt_socios_pj_inaptos:</b> valor inteiro, quantidade de parceiros PJ que são INAPTA no RF.</li>
    <li><b>vl_idade_media_socios_pj:</b> valor flutuante, idade média dos parceiros PJ.</li>
    <li><b>vl_idade_maxima_socios_pj:</b> valor flutuante, idade máxima dos parceiros PJ.</li>
    <li><b>vl_idade_minima_socios_pj:</b> valor flutuante, idade mínima dos parceiros PJ.</li>
    <li><b>qt_coligados:</b> valor inteiro, quantidade de empresas conectadas</li>
    <li><b>qt_socios_coligados:</b> valor inteiro, soma de parceiros de empresas conectadas</li>
    <li><b>qt_coligados_matriz:</b> valor inteiro, quantidade de empresas conectadas que são uma empresa matricial</li>
    <li><b>qt_coligados_ativo:</b> valor inteiro, quantidade de empresas conectadas consideradas ativas de acordo com o IRS - receita federal</li>
    <li><b>qt_coligados_baixada:</b> valor inteiro, quantidade de empresas conectadas consideradas fechadas de acordo com o IRS</li>
    <li><b>qt_coligados_inapta:</b> valor inteiro, quantidade de empresas conectadas consideradas inaptas / impróprias de acordo com o IRS</li>
    <li><b>qt_coligados_suspensa:</b> valor inteiro, quantidade de empresas conectadas que são consideradas suspensas de acordo com o IRS</li>
    <li><b>qt_coligados_nula:</b> valor inteiro, quantidade de empresas conectadas que são consideradas empresas nulos de acordo com o IRS</li>
    <li><b>idade_media_coligadas:</b> float, idade média (meses) de empresas conectadas</li>
    <li><b>idade_maxima_coligadas:</b> float, idade máxima (meses) de empresas conectadas</li>
    <li><b>idade_minima_coligadas:</b> float, idade mínima (meses) das empresas conectadas</li>
    <li><b>coligada_mais_nova_ativa:</b> flutuador, idade (meses) da colagem mais jovem que está ativa</li>
    <li><b>coligada_mais_antiga_ativa:</b> flutuador, idade (meses) da colagem mais antiga que está ativa</li>
    <li><b>idade_media_coligadas_ativas:</b> float, idade média (meses) de empresas conectadas ativas</li>
    <li><b style="color:#EB8958">coligada_mais_nova_baixada:</b> flutuador, idade (meses) da sua colagem fechada</li>
    <li><b style="color:#EB8958;">coligada_mais_antiga_baixada:</b> flutuador, idade (meses) da colagem fechada mais antiga</li>
    <li><b style="color:#EB8958;">idade_media_coligadas_baixadas:</b> flutuador, média (meses) de colagem fechada</li>
    <li><b>qt_coligados_sa:</b> valor inteiro, quantidade de empresas conectadas que possuem o termo SA no final do nome da empresa</li>
    <li><b>qt_coligados_me:</b> valor inteiro, quantidade de empresas conectadas que possuem o termo ME no final do nome da empresa</li>
    <li><b>qt_coligados_mei:</b> valor inteiro, quantidade de empresas conectadas que possuem o termo MEI no final do nome da empresa</li>
    <li><b>qt_coligados_ltda:</b> valor inteiro, quantidade de empresas conectadas que possuem o termo LTDA no final do nome da empresa</li>
    <li><b>qt_coligados_epp:</b> valor inteiro, quantidade de empresas conectadas que possuem o termo EPP no final do nome da empresa</li>
    <li><b>qt_coligados_norte:</b> valor inteiro, quantidade de empresas conectadas localizadas na região norte do país</li>
    <li><b>qt_coligados_sul:</b> valor inteiro, quantidade de empresas conectadas localizadas na região sul do país</li>
    <li><b>qt_coligados_nordeste:</b> valor inteiro, quantidade de empresas conectadas localizadas na região nordeste do país</li>
    <li><b>qt_coligados_centro:</b> valor inteiro, quantidade de empresas conectadas localizadas na região centro-oeste (centro-oeste) do país</li>
    <li><b>qt_coligados_sudeste:</b> valor inteiro, quantidade de empresas conectadas localizadas no sudeste do país</li>
    <li><b>qt_coligados_exterior:</b> valor inteiro, quantidade de empresas conectadas localizadas no exterior no país</li>
    <li><b>qt_ufs_coligados:</b> valor inteiro, quantidade de estados distintos em que as empresas conectadas estão localizadas</li>
    <li><b>qt_regioes_coligados:</b> valor inteiro, quantidade de ufs distintos em que as empresas conectadas estão localizadas</li>
    <li><b>qt_ramos_coligados:</b> valor inteiro, quantidade de ramos econômicos distintos de empresas conectadas</li>
    <li><b>qt_coligados_industria:</b> valor inteiro, quantidade de empresas conectadas do setor industrial</li>
    <li><b>qt_coligados_agropecuaria:</b> valor inteiro, quantidade de empresas conectadas do setor de agronegócio</li>
    <li><b>qt_coligados_comercio:</b> valor inteiro, quantidade de empresas conectadas</li>
    <li><b>qt_coligados_serviço:</b> valor inteiro, quantidade de empresas conectadas do setor de serviços</li>
    <li><b>qt_coligados_ccivil:</b> valor inteiro, quantidade de empresas conectadas do setor de construção</li>
    <li><b>qt_funcionarios_coligados:</b> valor inteiro, quantidade de funcionários das empresas conectadas</li>
    <li><b>qt_funcionarios_coligados_gp:</b> valor inteiro, quantidade de funcionários de empresas conectadas e de suas subsidiárias</li>
    <li><b>media_funcionarios_coligados_gp:</b> valor numérico, média de funcionários das empresas conectadas e de suas subsidiárias</li>
    <li><b>max_funcionarios_coligados_gp:</b> valor numérico, número máximo de funcionários de empresas conectadas e suas subsidiárias</li>
    <li><b>min_funcionarios_coligados_gp:</b> valor numérico, número mínimo de funcionários de empresas conectadas e suas subsidiárias</li>
    <li><b>vl_folha_coligados:</b> valor numérico, soma do valor da folha de pagamento de empresas conectadas</li>
    <li><b>media_vl_folha_coligados:</b> valor numérico, média do valor da folha de pagamento de empresas conectadas</li>
    <li><b>max_vl_folha_coligados:</b> valor numérico, máximo do valor da folha de pagamento de empresas conectadas</li>
    <li><b>min_vl_folha_coligados:</b> valor numérico, mínimo do valor da folha de pagamento de empresas conectadas</li>
    <li><b>vl_folha_coligados_gp:</b> valor numérico, soma do valor da folha de pagamento das empresas conectadas e de suas subsidiárias</li>
    <li><b>media_vl_folha_coligados_gp:</b> valor numérico, média do valor da folha de pagamento das empresas conectadas e de suas subsidiárias</li>
    <li><b>max_vl_folha_coligados_gp:</b> valor numérico, máximo do valor da folha de pagamento das empresas conectadas e de suas subsidiárias</li>
    <li><b>min_vl_folha_coligados_gp:</b> valor numérico, mínimo do valor da folha de pagamento das empresas conectadas e de suas subsidiárias</li>
    <li><b>faturamento_est_coligados:</b> valor numérico, soma da receita estimada de empresas conectadas</li>
    <li><b>media_faturamento_est_coligados:</b> valor numérico, média da receita estimada de empresas conectadas</li>
    <li><b>max_faturamento_est_coligados:</b> valor numérico, máximo da receita estimada de empresas conectadas</li>
    <li><b>min_faturamento_est_coligados:</b> valor numérico, mínimo da receita estimada de empresas conectadas</li>
    <li><b>faturamento_est_coligados_gp:</b> valor numérico, soma da receita estimada de empresas conectadas e de suas subsidiárias</li>
    <li><b>media_faturamento_est_coligados_gp:</b> valor numérico, média da receita estimada de empresas conectadas e de suas subsidiárias</li>
    <li><b>max_faturamento_est_coligados_gp:</b> valor numérico, valor máximo da receita estimada das empresas conectadas e de suas subsidiárias</li>
    <li><b>min_faturamento_est_coligados_gp:</b> valor numérico, valor mínimo da receita estimada das empresas conectadas e de suas subsidiárias</li>
    <li><b>total_filiais_coligados:</b> valor inteiro, quantidade de subsidiárias de empresas conectadas</li>
    <li><b>media_filiais_coligados:</b> valor numérico, média de subsidiárias de empresas conectadas</li>
    <li><b>max_filiais_coligados:</b> valor numérico, número máximo de subsidiárias de empresas conectadas</li>
    <li><b>min_filiais_coligados:</b> valor numérico, número mínimo de subsidiárias de empresas conectadas</li>
    <li><b>qt_coligados_atividade_alto:</b> valor inteiro, quantidade de empresas conectadas com classificador de nível de atividade estimado igual a 'ALTO'</li>
    <li><b>qt_coligados_atividade_medio:</b> valor inteiro, quantidade de empresas conectadas com classificador de nível de atividade estimado igual a 'Medio'</li>
    <li><b>qt_coligados_atividade_baixo:</b> valor inteiro, quantidade de empresas conectadas com classificador de nível de atividade estimado igual a 'Baixo'</li>
    <li><b>qt_coligados_atividade_mt_baixo:</b> valor inteiro, quantidade de empresas conectadas com classificador de nível de atividade estimado igual a 'Muito Baixo'</li>
    <li><b>qt_coligados_atividade_inativo:</b> valor inteiro, quantidade de empresas conectadas com classificador de nível de atividade estimado igual a 'INATIVO'</li>
    <li><b>qt_coligadas:</b> número de empresas vinculadas, ou seja, empresas em que um dos principais proprietários (CJNPJ referenciado) tem participação.</li>
    <li><b>sum_faturamento_estimado_coligadas:</b> a soma das estimativas de allcoligadas</li>
    <li><b>de_faixa_faturamento_estimado:</b> valor do caractere, classe da receita estimada da empresa</li>
    <li><b>de_faixa_faturamento_estimado_grupo:</b> valor do caractere, classe da soma da receita estimada da empresa matriz e filiais</li>
    <li><b>vl_faturamento_estimado_aux:</b> valor numérico, valor da receita estimada</li>
    <li><b>vl_faturamento_estimado_grupo_aux:</b>  valor numérico, soma da receita estimada para a empresa matriz e as filiais</li>
    <li><b>qt_ex_funcionarios:</b> valor inteiro, número de funcionários que deixaram a empresa.</li>
    <li><b>qt_funcionarios_grupo:</b> valor inteiro, quantidade de funcionários ativos considerando as filiais e a matriz.</li>
    <li><b>percent_func_genero_masc:</b> valor numérico, participação de funcionários do sexo masculino na empresa</li>
    <li><b>percent_func_genero_fem:</b> valor numérico, participação de funcionárias na empresa</li>
    <li><b>idade_ate_18:</b> valor inteiro, quantidade de funcionários com menos de 18 anos de idade.</li>
    <li><b>idade_de_19_a_23:</b> valor inteiro, quantidade de funcionários com idade entre 19 e 23 anos</li>
    <li><b>idade_de_24_a_28:</b> valor inteiro, quantidade de funcionários com idade entre 24 e 28 anos.</li>
    <li><b>idade_de_29_a_33:</b> valor inteiro, quantidade de funcionários com idade entre 29 e 33 anos.</li>
    <li><b>idade_de_34_a_38:</b> valor inteiro, quantidade de funcionários com idade entre 34 e 38 anos.</li>
    <li><b>idade_de_39_a_43:</b> valor inteiro, quantidade de funcionários com idade entre 39 e 43 anos.</li>
    <li><b>idade_de_44_a_48:</b> valor inteiro, quantidade de funcionários com idade entre 44 e 48 anos.</li>
    <li><b>idade_de_49_a_53:</b> valor inteiro, quantidade de funcionários com idade entre 49 e 53 anos.</li>
    <li><b>idade_de_54_a_58:</b> valor inteiro, quantidade de funcionários com idade entre 54 e 58 anos</li>
    <li><b>idade_acima_de_58:</b> valor inteiro, quantidade de funcionários com idade acima</li>
    <li><b>grau_instrucao_macro_analfabeto:</b> valor inteiro, quantidade de funcionários em posições sem necessidade de escolaridade.</li>
    <li><b>grau_instrucao_macro_escolaridade_fundamental:</b> valor inteiro, quantidade de empregados em posições com o nível fundamental exigido (completo e incompleto)</li>
    <li><b>grau_instrucao_macro_escolaridade_media:</b> valor inteiro, soma de grau_instrucao_medio_incompl e grau_instrucao_medio_compl</li>
    <li><b>grau_instrucao_macro_escolaridade_superior:</b> valor inteiro, soma de grau_instrucao_sup_incompl, grau_instrucao_sup_compl, grau_instrucao_mestrado e grau_instrucao_doutorado</li>
    <li><b style="color:red;">grau_instrucao_macro_desconhecido:</b> valor inteiro, o mesmo que grau_instrucao_desconhecido</li>
    <li><b>total:</b> valor inteiro, número real de funcionários da empresa</li>
    <li><b>meses_ultima_contratacaco:</b>  numérico, meses desde a última contratação.</li>
    <li><b>qt_admitidos_12meses:</b> número inteiro, quantidade de funcionários contratados nos últimos 12 meses</li>
    <li><b>qt_desligados_12meses:</b> número inteiro, quantidade de funcionários demitidos nos últimos 12 meses</li>
    <li><b>qt_desligados:</b> número inteiro, quantidade de funcionários demitidos pela empresa</li>
    <li><b>qt_admitidos:</b> número inteiro, quantidade de funcionários contratados pela empresa</li>
    <li><b>media_meses_servicos_all:</b> numérico, número médio de meses trabalhados por todos os funcionários da empresa (ativos e demitidos)</li>
    <li><b>max_meses_servicos_all:</b> numérico, número máximo de meses trabalhados por todos os funcionários da empresa (ativos e demitidos)</li>
    <li><b>min_meses_servicos_all:</b> numérico, número mínimo de meses trabalhados por todos os funcionários da empresa (ativos e demitidos)</li>
    <li><b>media_meses_servicos:</b> numérico, número médio de meses trabalhados pelos funcionários ativos da empresa</li>
    <li><b>max_meses_servicos:</b> numérico, número máximo de meses trabalhados pelos funcionários ativos da empresa</li>
    <li><b>min_meses_servicos:</b> numérico, número mínimo de meses trabalhados pelos funcionários ativos da empresa</li>
    <li><b>qt_funcionarios:</b> número inteiro, quantidade de empregados reais</li>
    <li><b>qt_funcionarios_12meses:</b> número inteiro, quantidade de funcionários ativos 12 meses antes</li>
    <li><b>qt_funcionarios_24meses:</b> número inteiro, quantidade de funcionários ativos 24 meses antes</li>
    <li><b>tx_crescimento_12meses:</b> número inteiro, crescimento de funcionários em relação ao número de funcionários 12 meses antes</li>
    <li><b>tx_crescimento_24meses:</b> número inteiro, crescimento de funcionários em relação ao número de funcionários 24 meses antes</li>
    <li><b>tx_rotatividade:</b> número inteiro, mede a estabilidade geral dos funcionários considerando a contratação e demissões e o número real de funcionários.</li>
    <li><b>qt_filiais:</b> auto-explicativo.</li>
</ol>

### Filtrando colunas "relevantes" do market

In [19]:
market_clean = filter_columns(market)

In [20]:
market_clean = fillna_columns(market_clean)

In [21]:
exploracao(market_clean)

,coluna,tipos,qtd_null,percent_null
de_nivel_atividade,de_nivel_atividade,object,0,0.0
de_ramo,de_ramo,object,0,0.0
vl_total_veiculos_leves_grupo,vl_total_veiculos_leves_grupo,float64,0,0.0
vl_faturamento_estimado_grupo_aux,vl_faturamento_estimado_grupo_aux,float64,0,0.0
vl_faturamento_estimado_aux,vl_faturamento_estimado_aux,float64,0,0.0
sg_uf,sg_uf,object,0,0.0
setor,setor,object,0,0.0
qt_filiais,qt_filiais,int64,0,0.0
nm_segmento,nm_segmento,object,0,0.0
nm_meso_regiao,nm_meso_regiao,object,0,0.0


In [22]:
market_clean.head()

,de_nivel_atividade,de_ramo,de_saude_tributaria,fl_antt,fl_ltda,fl_matriz,fl_me,fl_mei,fl_optante_simei,fl_optante_simples,fl_passivel_iss,fl_rm,fl_sa,fl_simples_irregular,fl_spa,fl_st_especial,id,idade_empresa_anos,natureza_juridica_macro,nm_meso_regiao,nm_segmento,qt_filiais,setor,sg_uf,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,vl_total_veiculos_leves_grupo,vl_total_veiculos_pesados_grupo
0,ALTA,INDUSTRIA DA CONSTRUCAO,VERDE,False,False,True,False,False,False,True,True,True,False,False,False,False,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,14.457534,ENTIDADES EMPRESARIAIS,LESTE POTIGUAR,CONSTRUCAO,0,CONSTRUÇÃO CIVIL,RN,3132172.8,3132172.8,0.0,0.0
1,BAIXA,SERVICOS DE ALOJAMENTO/ALIMENTACAO,CINZA,False,False,True,False,True,False,False,True,True,False,False,False,False,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,1.463014,OUTROS,CENTRO NORTE PIAUIENSE,ALOJAMENTO E ALIMENTACAO,0,SERVIÇO,PI,210000.0,210000.0,0.0,0.0
2,MEDIA,"TRANSPORTE, ARMAZENAGEM E CORREIO",AMARELO,False,False,True,False,True,True,True,True,True,False,False,False,False,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,7.093151,OUTROS,CENTRO AMAZONENSE,TRANSPORTE ARMAZENAGEM E CORREIO,0,SERVIÇO,AM,50000.0,50000.0,0.0,0.0
3,MEDIA,SERVICOS DIVERSOS,AMARELO,False,False,True,False,False,False,True,True,True,False,False,False,False,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,6.512329,OUTROS,CENTRO AMAZONENSE,OUTRAS ATIVIDADES DE SERVICOS,0,SERVIÇO,AM,210000.0,210000.0,0.0,0.0
4,ALTA,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",VERDE,False,False,True,False,False,False,True,True,True,False,False,False,False,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,3.200000,OUTROS,LESTE POTIGUAR,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,0,SERVIÇO,RN,210000.0,210000.0,0.0,0.0


# Criar modelo de baseline

In [23]:
market_clean = market_clean.set_index('id')
market_clean.head()

,de_nivel_atividade,de_ramo,de_saude_tributaria,fl_antt,fl_ltda,fl_matriz,fl_me,fl_mei,fl_optante_simei,fl_optante_simples,fl_passivel_iss,fl_rm,fl_sa,fl_simples_irregular,fl_spa,fl_st_especial,idade_empresa_anos,natureza_juridica_macro,nm_meso_regiao,nm_segmento,qt_filiais,setor,sg_uf,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,vl_total_veiculos_leves_grupo,vl_total_veiculos_pesados_grupo
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
a6984c3ae395090e3bee8ad63c3758b110de096d5d819583a784a113726db849,ALTA,INDUSTRIA DA CONSTRUCAO,VERDE,False,False,True,False,False,False,True,True,True,False,False,False,False,14.457534,ENTIDADES EMPRESARIAIS,LESTE POTIGUAR,CONSTRUCAO,0,CONSTRUÇÃO CIVIL,RN,3132172.8,3132172.8,0.0,0.0
6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c47fdef50b33f4f56f05,BAIXA,SERVICOS DE ALOJAMENTO/ALIMENTACAO,CINZA,False,False,True,False,True,False,False,True,True,False,False,False,False,1.463014,OUTROS,CENTRO NORTE PIAUIENSE,ALOJAMENTO E ALIMENTACAO,0,SERVIÇO,PI,210000.0,210000.0,0.0,0.0
4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02edc5147bc2a2535b08,MEDIA,"TRANSPORTE, ARMAZENAGEM E CORREIO",AMARELO,False,False,True,False,True,True,True,True,True,False,False,False,False,7.093151,OUTROS,CENTRO AMAZONENSE,TRANSPORTE ARMAZENAGEM E CORREIO,0,SERVIÇO,AM,50000.0,50000.0,0.0,0.0
3348900fe63216a439d2e5238c79ddd46ede454df7b9d8c24ac33eb21d4b21ef,MEDIA,SERVICOS DIVERSOS,AMARELO,False,False,True,False,False,False,True,True,True,False,False,False,False,6.512329,OUTROS,CENTRO AMAZONENSE,OUTRAS ATIVIDADES DE SERVICOS,0,SERVIÇO,AM,210000.0,210000.0,0.0,0.0
1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4b102c769f7611ab29f,ALTA,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",VERDE,False,False,True,False,False,False,True,True,True,False,False,False,False,3.200000,OUTROS,LESTE POTIGUAR,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,0,SERVIÇO,RN,210000.0,210000.0,0.0,0.0


### Verificando os tipos de dados das colunas do dataframe

In [24]:
features_numericas = list(market_clean.select_dtypes(include=['float', 'int']).columns)

In [25]:
featues_categoricas = list(market_clean.select_dtypes(include='object'))

## Criação do pipeline para deixar o code mais clean

In [26]:
num_pipeline = Pipeline(steps=[
    ("log", FunctionTransformer(np.log1p)),
])

cat_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore')),
])


column_transformer = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, features_numericas),
        ('cat', cat_pipeline, featues_categoricas)
    ],
    remainder='passthrough'
)

pipeline_transformation = Pipeline(steps=[
    ('col', column_transformer),
    ('pca', PCA(n_components=0.95))
])

In [27]:
X = pipeline_transformation.fit_transform(market_clean)

In [28]:
nn_model = NearestNeighbors(n_neighbors=5, algorithm='ball_tree', metric='minkowski', n_jobs=-1).fit(X)

In [29]:
train, test = train_test_split(port_1[['id']], test_size=0.3, random_state=0)

In [30]:
pd_p = pd.merge(port_1[['id']], market_clean, on='id', how='inner')

In [31]:
X_pf = pipeline_transformation.transform(pd_p.drop(columns=['id']))

In [63]:
distancia, indices = nn_model.kneighbors(X_pf, n_neighbors=5)

In [64]:
indices

array([[170251, 272908,  96495, 221802,    369],
       [   604,  47308, 231754, 368594, 165612],
       [  1703, 216723, 287060, 185897, 422637],
       ...,
       [461483, 272759, 228772, 177207, 151132],
       [461978,  53925, 232618,  42826, 271627],
       [462190, 169738, 452089, 151397, 283533]])

In [65]:
distancia

array([[8.91034596e-15, 8.91034596e-15, 8.91034596e-15, 8.91034596e-15,
        8.91034596e-15],
       [1.41572695e-14, 4.63498490e-01, 5.45899648e-01, 5.81564340e-01,
        5.92989295e-01],
       [1.69152160e-14, 6.92984434e-03, 1.11719444e-02, 2.28522309e-02,
        2.86339876e-02],
       ...,
       [5.71221656e-15, 2.30743370e-03, 3.28700900e-03, 3.95890790e-03,
        4.04155415e-03],
       [1.01125741e-14, 1.01125741e-14, 1.09267922e-02, 1.24389173e-02,
        1.63451803e-02],
       [1.72884902e-14, 5.41540330e-04, 1.08278384e-03, 1.62640514e-03,
        1.62640514e-03]])

In [67]:
mask = ~np.isin(indices, set(port_1[['id']]))
distancia = distancia[mask]
indices = indices[mask]

In [68]:
data = set(zip(distancia, indices))
results = pd.DataFrame(data, columns=['distances', 'indices'])

In [70]:
results.indices.value_counts()

7822      3
218889    3
330067    2
62299     2
177029    2
         ..
421297    1
207532    1
4425      1
340649    1
228519    1
Name: indices, Length: 2763, dtype: int64

In [71]:
top_indices = results.groupby('indices')\
                         .agg(count=('indices', 'count'), 
                              distances=('distances', 'min'))\
                         .sort_values(['count', 'distances'], ascending=[False, True])\
                         .index.values

In [72]:
top_indices

array([  7822, 218889, 272010, ..., 238273, 123315, 111367])

In [80]:
ids = market_clean.iloc[top_indices].index

In [81]:
test.index.isin(ids).sum()

0

In [42]:
lista_simples_indices_knn = lista_simples(list(indices))

In [52]:
lista_simples_indices_knn

[array([170251, 272908,  96495, 221802,    369]),
 array([   604,  47308, 231754, 368594, 165612]),
 array([  1703, 216723, 287060, 185897, 422637]),
 array([  2521, 171410, 439424,  94655, 152601]),
 array([  5788, 261919, 451761,  75763,  32632]),
 array([  6369, 400658, 258580, 298473, 258462]),
 array([  7822, 218889, 330067, 275504,  62299]),
 array([200587,   8949, 136627,  78893,  61858]),
 array([  9272,  61817, 171311, 265882,  90099]),
 array([  9946,  80404, 288974,  37212, 371046]),
 array([ 11051,  18947, 270912, 273200, 248150]),
 array([ 12417,  37032,   7119, 375085, 392019]),
 array([ 12619, 150015, 349931, 427381, 142691]),
 array([ 13195,  43060, 340533, 447490, 272506]),
 array([ 15365, 150072, 170208, 449197,  87612]),
 array([181510,  50551, 407122,  16609,  40889]),
 array([ 16874, 182528,  90083, 338229, 252157]),
 array([ 21394, 234706,  89518,  80837, 422457]),
 array([ 22351, 409403, 106687, 235932,  86095]),
 array([ 24371, 221621, 377521, 159816,  13986]),


In [62]:
(lista_simples([l.tolist() for l in lista_simples_indices_knn]))

AttributeError: 'list' object has no attribute 'value_counts'

In [ ]:
market_clean.dtypes.value_counts()

In [ ]:
market_clean.head()

In [ ]:
le = LabelEncoder()

In [ ]:
market_clean.dtypes.value_counts()

In [ ]:
df_encode = encoding_obj(market_clean, le)

In [ ]:
df_encode.dtypes.value_counts()

In [ ]:
df_encode.shape

In [ ]:
df_encode.columns

## Criando relatório de EDA com a lib Sweet Viz e Pandas Profilling

In [ ]:
# my_report = sv.analyze(market)
# my_report.show_html(filepath='eda-enconding.html')

In [ ]:
# profile = ProfileReport(market, minimal=True)
# profile.to_file("pp-market.html")

In [ ]:
pca = PCA()

### Criando instância do NearestNeighbors

In [ ]:
knn = 

### Treinando modelo

In [ ]:
%%time
knn.fit(df_encode)

### Tratar portfólio da empresa 2

In [ ]:
port_2.shape

### Filtrar pelas colunas escolhidas, dropna e seta id como index

In [ ]:
port_2_clean = filter_columns(port_2)

In [ ]:
for col in port_2_clean.columns:
    if 'fl_' in col:
        if {'SIM', 'NAO'} <= set(port_2_clean[col]):
            port_2_clean[col] = (port_2_clean[col]=='SIM')
        port_2_clean[col] = port_2_clean[col].fillna(False)
    elif 'vl_' in col or 'media_' in col or 'max_' in col or 'min_' in col or 'percent_' in col: 
        mean = port_2_clean[col].mean()
        port_2_clean[col] = port_2_clean[col].fillna(mean)
    elif port_2_clean[col].dtype=='object':
        port_2_clean[col] = port_2_clean[col].fillna('SEM INFORMACAO' if 'de_' in col else 'OUTRA')
    elif 'dt_' in col:
        pass
    else:
        port_2_clean[col] = port_2_clean[col].fillna(0).astype('int')

In [ ]:
#port_2_clean = port_2_clean.dropna()
port_2_clean = port_2_clean.set_index('id')
port_2_clean.head()

In [ ]:
df_encode.shape

In [ ]:
port_2_clean.shape

In [ ]:
#port_2_clean[df_encode.columns]

### Realiza o encoding das features

In [ ]:
port_2_encode = encoding_obj(port_2_clean[df_encode.columns], le)

In [ ]:
port_2_encode

### Divide o dataframe da empresa 2 em treino e teste

In [ ]:
X_train, X_test = treino_teste_port(port_2_encode)

In [ ]:
print(f"Treino tem o shape {X_train.shape}")
print(f"Teste tem o shape {X_test.shape}")

### Busca os vizinhos mais próximos para cada cliente do treino da empresa 2 e monta uma lista com os indices

In [ ]:
%%time
lista_indices_knn =  []
for i in range(X_train.shape[0]):
    indices = knn.kneighbors([X_train.loc[X_train.index[i]]], n_neighbors=10, return_distance=False)
    lista_indices_knn.append(list(indices[0]))

### A lista acima se torno uma lista de listas, a função abaixo transforma em um lista simples

In [ ]:
lista_simples_indices_knn = lista_simples(lista_indices_knn)

### Essa função cria um dataframe com as empresas vizinhas do treino da empresa 2

In [ ]:
def criar_df_knn(lista_simples_indices_knn, df_modelo_treino):
    df_knn = pd.DataFrame(columns=df_modelo_treino.columns)
    for i in range(len(lista_simples_indices_knn)):
        df_knn = df_knn.append(df_modelo_treino.iloc[lista_simples_indices_knn[i]])

    df_knn.index.name = 'id'
    return df_knn

In [ ]:
df_knn = criar_df_knn(lista_simples_indices_knn, df_encode)

In [ ]:
ids_pred = list(df_knn.index.drop_duplicates(keep='first'))

In [ ]:
X_test.shape

In [ ]:
X_test.index.isin(ids_pred).sum()

In [ ]:
ids_true = X_test[X_test.index.isin(ids_pred)]
ids_true = list(ids_true.index)

In [ ]:
len(ids_true), len(ids_pred)

In [ ]:
len(X_test.index) - 

In [ ]:
probabilidade(len(ids_true), X_test.shape[0])

### Probalidade de acerto

#### excluindo as colunas que estao na função filter_columns
- port 2 - 77.09%
- port 1 - 0
- port 3 - 71.7%

#### sem excluir as colunas que estao na função filter_columns
- port 2 - 77.97%
- port 1 - 0
- port 3 - 70.75%

### Acima verifica de quantas empresas vizinhas foram "recomendadas de forma correta"

## trocar ideia com o chrys sobre
- recomendei 1735 empresas, na base de teste tem 149
- deu match em 29 casos
- quando filtro pelas 149 "primeiras", deu match as mesmas 29
- ""ou seja se eu recomenda-se apenas 149, iria aumentar minha acuracia""

## Novidades:
- config assim deu bom: (n_neighbors=5, algorithm='ball_tree', metric='minkowski', n_jobs=-1) de 199 achou 99
- config assim deu bom: (n_neighbors=5, algorithm='ball_tree', metric='euclidean', n_jobs=-1) de 199 achou 99
- config assim deu bom: (n_neighbors=5, algorithm='kd_tree', metric='euclidean', n_jobs=-1) de 199 achou 99

In [ ]:
len(set(X_test.index) & set(ids_true)) / len(set(X_test.index)) * 100

# ANOTAÇÕES

#### separa o portfolio em treino e teste. Passa o treino como argumento e a partir daí vai ser gerado os leads, e verifica quantos dos leads recomendados existem no teste

### Passos para o feriado
- aplicar pca e criar modelo a partir da pca

# Abordagem anotada no boleto
- Treinar market com KNN
- Dividir um portfólio em treino e teste
- Pegar os k vizinhos mais próximos para cada empresa do portfólio de treino
- Verificar quantos desses vizinhos estão no portfólio de teste

# Passos para sexta, pegar as distancias do vizinhos e ordenar pela menor distancia